In [1]:
# Imports
import dask.dataframe as dd
from dask.distributed import Client

In [2]:
# Create a distibuted client for parallel computation
client = Client(n_workers=16) # Since we are using AWS SageMaker with 16 cores

youtube_df = dd.read_csv('/home/ec2-user/SageMaker/Youtube*.csv')

In [3]:
# Check the number of columns and rows in the dataframe
print(f"Number of columns in dataframe is {len(youtube_df.columns)}")
print(f"Number of rows in dataframe is {len(youtube_df)}")

Number of columns in dataframe is 5
Number of rows in dataframe is 1956


In [4]:
youtube_df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [5]:
(spam, notspam) = youtube_df.CLASS.value_counts().compute()
print(f"spam count is {spam} and legitimate comments count is {notspam}")

spam count is 1005 and legitimate comments count is 951


In [6]:
# Seperating out data with respect to spam and nonspam
spamdf = youtube_df[youtube_df.CLASS == 1].compute()
nonspamdf = youtube_df[youtube_df.CLASS == 0].compute()

In [7]:
# Computing the count of 'check' in spam and nonspam data
checkwordcount_spam = 0
for comment in spamdf.CONTENT:
    if "check" in comment.lower():
        checkwordcount_spam += 1

print(f"Number of comments containing 'check' in spams is {checkwordcount_spam}")

checkwordcount_nonspam = 0
for comment in nonspamdf.CONTENT:
    if "check" in comment.lower():
        checkwordcount_nonspam += 1

print(f"Number of comments containing 'check' in spams is {checkwordcount_nonspam}")

Number of comments containing 'check' in spams is 461
Number of comments containing 'check' in spams is 19


In [8]:
# To score a 3, do extra work, such as creating the Dask Distributed Client, or creating a visualization with this dataset.

# I have added below code to complete this stretch goal already.
# client = Client(n_workers=16) # Since we are using AWS SageMaker with 16 cores

### Big data options

I would like to first discuss about the platform.

**AWS Sagemaker:**

AWS Sagemaker is a powerful platform for working with big dataset which needs very high computational resources (CPU and RAM to be specific). Our exisiting PCs/laptops have limitation with respect to computational resource. For example my laptop has 8 core CPU and 32 GB RAM capacity. If the data which needs to be processed is huge and takes lot of time to process assume 10 GB of data, then the capacity on my existing laptop is not sufficient. (As per guidelines usually 10X amount of capacity is needed).

In this case, I can make use of AWS Sagemaker which gives me an option to **Scale-Up** my capacity and work with huge data sets. Advantage of this approach is that there is no need for actually worrying about partitioning the data and combining it together later since we are running on a single system. 

**AWS EMR/Databricks**

As an alternative to **Scale-Up** which usually needs bigger and powerful computational resource on a single node, we can use the approach of **Scale-Out** using similar or almost similar nodes with comparable computational resources. The major disadvantage of **Scale-Up** approach is there is a physical and economical limitation with respect to how much we can Scale-Up computationally. In simpler words, there are limitation on how big the Super Computers can be and also they are very very expensive.

So with **Scale-Out** approach, we can have nodes having a lower computational resources but use them together is a managed way to distribute the work among them logically and process the data. The technology required for such distributed computing (like Hadoop, Spark...) are existing and can be used. So here instead of going for expensive AWS Sagemaker, we can use nodes of lower capacity like AWS EMR/Databricks. Some key disadvantage of this approch is fault tolerance/reducancy needs to be considered. Also for distributing the computation and managing the task their would be additional overhead.


Next I would like to first discuss about the libraries.

**Numba**

Numba is a just-in-time compiler for Python that works best on code that uses NumPy arrays and functions, and loops. Numba library optimises the execution of instructions using just-in-time compiling and can work with existing python code.

However Numba is suited best to work with Numpy based code and has limitation with respect to other libraries like Pandas.
Ideal for initial step of **Step-Up** optimization.

**Dask**

Dask is a flexible library for parallel computing in Python.

Dask is composed of two parts:

1. Dynamic task scheduling optimized for computation and optimized for interactive computational workloads.
2 . “Big Data” collections like parallel arrays, dataframes, and lists that extend common interfaces like NumPy, Pandas, or Python iterators to larger-than-memory or distributed environments. These parallel collections run on top of dynamic task schedulers.

Ideal for **Step-Up** optimization and also for **Step-Out** approaches where we can distribute the load across multiple nodes. However, for better **Step-Out** optimization MapReduce/Spark would be better alternative.

The interface are very similar to Pandas and Sci-kit learn and needs minimal code changes.

**MapReduce/Spark**

As mentioned above, both MapReduce/Spark are developed to support **Step-Out** based computation and is best suited for Big Data processing which usually spans across multiple noded. The underlying computation overhead like distribution of data, fault tolerance, redundacy and consolidation is handled by these libraries.

To help development many of the common constructs from exisiting technology are supported on these libraries. For example Spark supports development on Scala, Python and Java. It also supports programming constructs similar to DataFrame and SQL.

Lastly let us consider the languages.

**Python**

Python is multipurpose programming langage which is extensively used in Data Science also. When considering **Scale-Up or Scale-Out** approaches Python has good libraries developed for both cases.

Numba and Dask can be used to optimize the python code on single nodes with just-in-time compilation and parallely processing.

Similarly, technology like Spark provides interface for developing code in Python and running ontop of Spark (PySpark) which is not very much optimized as compared to Scala Spark.

**SQL**

SQL is similar to Python. Spark supports interaction with data using SQL queries which is a very convinent way. In fact, native DataFrame or SQL based approach are evaluated by Spark platform similarly.

**Scala**

Scala is the 1st citizen in the world of Spark and in fact Spart is developed using Scala. So many of the interfaces in Spark are optimized for Scala language. Scala supports functional programming approch which is very useful for **Scale-Out** based approach.

**Java**

Java support on Spark is similar to Python.

**My Preference**

I would initially use python (Numpy/Pandas/Sci-Kit Learn) on single node system for small/medium data set. If the computation resource needed increases to a large data set, I would optimize it python code using Numba or Dask. Finally incase of Scale-Out is needed, I would develop the code using Scala/PySpark on Spark.